In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [2]:
df = pd.read_excel("output/distribucion_sociodemografica/distribucion_sociodemografica.xlsx")

In [47]:
locator = Nominatim(user_agent="javier", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

def location_info(x):
    try:
        data = locator.geocode(x).raw
        data_converted = pd.json_normalize(data).squeeze()

    except:
        data_converted = pd.Series(dtype='float64')

    return data_converted


In [51]:
regiones = list(df["region_pais"].unique())
regiones_unicas = pd.DataFrame(regiones, columns=["region_pais"])
objeto = regiones_unicas["region_pais"].apply(location_info)
objeto["lat"] = objeto["lat"].astype(float)
objeto["lon"] = objeto["lon"].astype(float)

union = pd.concat([regiones_unicas, objeto], axis=1)

In [68]:
comunas = list(df["comuna_region_pais"].unique())
comunas_unicas = pd.DataFrame(comunas, columns=["comuna_region_pais"])
objeto_comunas = comunas_unicas['comuna_region_pais'].apply(location_info)
union_comunas = pd.concat([comunas_unicas, objeto_comunas], axis=1)

In [78]:
with pd.ExcelWriter('localizacion.xlsx') as f:
    union.to_excel(f, sheet_name='regiones', index=False)
    union_comunas.to_excel(f, sheet_name='comunas', index=False)